# Step 3 - Calculate the ratio of roads suitable for bikes per district

PostGIS is more than just a database extension to store spatial data. It also allows for sophisticated data processing through many built-in spatial functions. In this section we will use PostGIS to process the road network and the district boundaries to get a new dataset that contains the ratio of roads suitable for bikes for each district.

> Note: This section is not intended to be a tutorial for SQL, PostgreSQL or PostGIS. If you are interested in understanding the basics, we recommend to checkout [this tutorial for basic SQL (using PostgreSQL dialect)](https://www.postgresqltutorial.com/) and [this tutorial to learn about the power of the spatial extension PostGIS](https://postgis.net/workshops/postgis-intro/). For the most fun hands-on experience, follow the tutorials using the Sandbox pgAdmin query interface and experiment with the data we just loaded.

**In a first step, let's describe what we want to do in plain text:**

_For each district, we want to derive the ratio of total length of roads suitable for biking within that district divided by the total length of all roads within that district. To that end, we need to find the parts of the road network which intersect with each district. This needs to be done twice for all the roads and only the roads suitable for biking (which have attribute velo=1). We then want to sum up their lengths respectively and derive the ratio._

**Let's now get a bit more technical using some PostgreSQL/PostGIS terminology:**
1. Use `ST_Length` to derive the length of roads from their geometry.
2. Get total road length per district using `GROUP BY`, `ST_Intersects` and `ST_Intersection`.
3. Get total length of roads suitable for bikes per district by adding a condition to the `WHERE` clause.
4. Combine the results of the previous steps using `WITH` and `JOIN` to derive the ratio of roads suitable for bikes.

***
**Your turn:**
- Open pgAdmin, connect to the PostGIS database and make sure the road network and district datasets are available.
- Open the pgAdmin query interface and follow along using the queries below.

![check tables](./story_images/check_tables.gif)

***
#### Use `ST_Length` to derive the length of roads from their geometry.

Return the length for each of the 40k road network features in meters:
```postgresql
SELECT ST_Length(zh_roads.geom)
FROM zh_roads;
```

Return the total length of all road network features in meters:
```postgresql
SELECT SUM(ST_Length(zh_roads.geom)) AS sum_total
FROM zh_roads;
```

***
#### Get total road length per district using `GROUP BY`, `ST_Intersects` and `ST_Intersection`.

SQL allows for arbitrary indentation, which is used in the following to make queries hopefully a little easier to understand. Return the length of all road network features in meters for each each district:
```postgresql
SELECT zh_districts.fid AS district_id, 
    SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS sum_total
FROM zh_districts, zh_roads
WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
GROUP BY zh_districts.fid;
```

***
#### Get total length of roads suitable for bikes per district using `WHERE` to filter for suitable roads.

Return the length of all road network features in meters for each district and are suitable for bikes:
```postgresql
SELECT zh_districts.fid AS district_id, 
    SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS sum_bike
FROM zh_districts, zh_roads
WHERE ST_Intersects(zh_districts.geom, zh_roads.geom) 
    AND zh_roads.bike=1
GROUP BY zh_districts.fid;
```

***
#### Combine the results of the previous steps using `WITH` and `JOIN` to derive the ratio of roads suitable for bikes.

As a next step we want to return both the total length per district as well as the length suitable for bikes. There are many ways to do this, but here we use a SQL feature called _Common Table Expression (CTE)_ that allows to create intermediary virtual tables that can be referenced in a subsequent query. Using the `WITH` keyword, we first create such intermediary tables from the two queries above called total_sum_table and bike_sum_table. These tables are joined with the zh_districts table on the feature id column (fid) in order to retrieve the district names and geometry. Run all the code below as a single query:
```postgresql
WITH sum_total_table AS (
    SELECT zh_districts.fid, 
        SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS sum_total_table
    FROM zh_districts, zh_roads
    WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
    GROUP BY zh_districts.fid
),
sum_bike_table AS (
    SELECT zh_districts.fid, 
        SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS sum_bike
    FROM zh_districts, zh_roads
    WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
        AND zh_roads.velo=1
    GROUP BY zh_districts.fid
)

SELECT
    zh_districts.fid AS district_id,
    zh_districts.stzname AS zonenname, 
    sum_total_table.sum_total_table AS sum_total,
    sum_bike_table.sum_bike AS sum_bike
FROM zh_districts
LEFT JOIN sum_total_table ON sum_total_table.fid = zh_districts.fid
LEFT JOIN sum_bike_table ON sum_bike_table.fid = zh_districts.fid;
```

***
Let's now add two more columns to the main `SELECT` clause which hold the ratio of road length suitable for bikes compared to total road length and the geometry of the districts:
```postgresql
WITH sum_total_table AS (
    SELECT zh_districts.fid, 
        SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS sum_total_table
    FROM zh_districts, zh_roads
    WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
    GROUP BY zh_districts.fid
),
sum_bike_table AS (
    SELECT zh_districts.fid, 
        SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS sum_bike
    FROM zh_districts, zh_roads
    WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
        AND zh_roads.velo=1
    GROUP BY zh_districts.fid
)

SELECT
    zh_districts.fid AS district_id,
    zh_districts.stzname AS zonenname, 
    sum_total_table.sum_total_table AS sum_total,
    sum_bike_table.sum_bike AS sum_bike,
    sum_bike_table.sum_bike/sum_total_table.sum_total_table AS ratio_suitable,
    zh_districts.geom AS geom
FROM zh_districts
LEFT JOIN sum_total_table ON sum_total_table.fid = zh_districts.fid
LEFT JOIN sum_bike_table ON sum_bike_table.fid = zh_districts.fid;
```

***
**Congratulations! With these few lines you performed some non-trivial spatial processing and aggregation in PostGIS! For each district you now have the ratio of how much of the roads are suitable for bikes.** Of course the underlying methodology and assumptions might be oversimplified and could be improved, but this is a solid first prototype!